In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras import backend as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.datasets import mnist
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop, Adam
import numpy as np
import matplotlib.pyplot as plt

In [17]:

from subprocess import check_output
print(check_output(["ls", "./data"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

test.csv
train.csv



### Load Train and Test Data:

In [18]:
# create the training & test sets, skipping the header row with [1:]
train = pd.read_csv("./data/train.csv")
print(train.shape)
train.head()

(42000, 785)


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
test= pd.read_csv("./data/test.csv")
print(test.shape)
test.head()

(28000, 784)


,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
X_train = train.drop(labels = ["label"],axis = 1) 
print(X_train)

       pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0           0       0       0       0       0       0       0       0       0   
1           0       0       0       0       0       0       0       0       0   
2           0       0       0       0       0       0       0       0       0   
3           0       0       0       0       0       0       0       0       0   
4           0       0       0       0       0       0       0       0       0   
5           0       0       0       0       0       0       0       0       0   
6           0       0       0       0       0       0       0       0       0   
7           0       0       0       0       0       0       0       0       0   
8           0       0       0       0       0       0       0       0       0   
9           0       0       0       0       0       0       0       0       0   
10          0       0       0       0       0       0       0       0       0   
11          0       0       

In [21]:
Y_train = train['label'].values.astype('int32') # only labels i.e targets digits
X_test = test
print(Y_train)

[1 0 1 ..., 7 6 9]


In [22]:
print("Shape of Training dataset Images:",X_train.shape)
print("Shape of Training dataset labels:",Y_train.shape)

Shape of Training dataset Images: (42000, 784)
Shape of Training dataset labels: (42000,)


In [23]:
K.set_image_dim_ordering("th")

In [24]:

X_train /= 255.0
X_test /= 255.0

In [25]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,1,28,28)
X_test = X_test.values.reshape(-1,1,28,28)

In [26]:
print(X_train.shape)
print(X_test.shape)

(42000, 1, 28, 28)
(28000, 1, 28, 28)


In [27]:
print("Training Sample Shape:",X_train.shape)
print("Testing Sample Shape:",X_test.shape)

Training Sample Shape: (42000, 1, 28, 28)
Testing Sample Shape: (28000, 1, 28, 28)


### Performing One Hot encoding:

In [28]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train, 10)

In [29]:
print(X_train.shape)
print(X_test.shape)

(42000, 1, 28, 28)
(28000, 1, 28, 28)


### Network and training

In [41]:
# network and training
NB_EPOCH = 20
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = Adam()
VALIDATION_SPLIT=0.2
IMG_ROWS, IMG_COLS = 28, 28 # input image dimensions
NB_CLASSES = 10 # number of outputs = number of digits
INPUT_SHAPE = (1, IMG_ROWS, IMG_COLS)


### LeNet Code in Keras:

In [42]:
#define the ConvNet
class LeNet:
    @staticmethod
    def build(input_shape, classes):
        model = Sequential() # Sequential model
     
        # CONV => RELU => POOL
        
        model.add(Conv2D(20, kernel_size=5, padding="same",input_shape=input_shape))
        # 20 ->  Convolutional kernel
        # kernel_size = 5 --> Specify the value of spatial dimensions.
        # padding ='same' --> We have an output same size as the input.
        # input_shape --> input shape of the image
        
        model.add(Activation("relu"))
        # Activation --> Relu
        
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # pool_size = (2,2) --> Represents factors in which the image is vertically and horizontally downscaled.
        
       
        # CONV => RELU => POOL
        model.add(Conv2D(50, kernel_size=5, border_mode="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        
        
        # Flatten => RELU layers   
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))
        
        # a softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

### Designing the neural network architecture:


In [43]:
# initialize the optimizer and model
model = LeNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)
model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
metrics=["accuracy"])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, kernel_size=5, padding="same")`


### Training:

In [ ]:
history = model.fit(X_train, Y_train,
batch_size=BATCH_SIZE, epochs=NB_EPOCH,
verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
33600/33600 [==============================] - 164s 5ms/step - loss: 0.2320 - acc: 0.9296 - val_loss: 0.0722 - val_acc: 0.9770
Epoch 2/20
33600/33600 [==============================] - 241s 7ms/step - loss: 0.0584 - acc: 0.9807 - val_loss: 0.0556 - val_acc: 0.9821
Epoch 3/20
33600/33600 [==============================] - 321s 10ms/step - loss: 0.0379 - acc: 0.9881 - val_loss: 0.0400 - val_acc: 0.9863
Epoch 4/20
33600/33600 [==============================] - 317s 9ms/step - loss: 0.0290 - acc: 0.9905 - val_loss: 0.0614 - val_acc: 0.9820
Epoch 5/20
33600/33600 [==============================] - 316s 9ms/step - loss: 0.0215 - acc: 0.9932 - val_loss: 0.0370 - val_acc: 0.9889
Epoch 6/20
33600/33600 [==============================] - 295s 9ms/step - loss: 0.0151 - acc: 0.9951 - val_loss: 0.0373 - val_acc: 0.9894
Epoch 7/20
33600/33600 [==============================] - 306s 9ms/step - loss: 0.0137 - acc: 0.9955 - val_loss: 0.0375 - 

In [40]:
predictions = model.predict_classes(X_test, verbose=0)
submissions=pd.DataFrame({'ImageId':list(range(1,len(predictions) + 1)), "Label": predictions})
submissions.to_csv("Final_Prediction_2.csv", index=False, header=True)